# Benchmarking models for solubility functions

This notebook benchmarks the models for solubility functions in VolFe where possible.

The following models are a constant and therefore are not benchmarked in this notebook:
- Models for solubility function of water (H2O): option = water, function = C_H2O
    - 'Basalt_Hughes24' Fig.S2 from Hughes et al. (2024)
    - 'Rhyolite_HughesIP' Fig.SX from Hughes et al. (in prep) based on data in Fig. 3 of Blank et al. (1993)
- Models for solubility function of hydrogen sulfide (H2S): option = hydrogen sulfide, function = C_H2S
    - 'Basalt_Hughes24' Fig.S6 from Hughes et al. (2024) based on experimental data Moune et al. (2009) and calculations in Lesne et al. (2011)
    - 'BasalticAndesite_Hughes24' Fig.S6 from Hughes et al. (2024) based on experimental data Moune et al. (2009) and calculations in Lesne et al. (2011)
- Models for solubility function of "X": option = species X solubility, function = C_X
    - 'Ar_Basalt_HughesIP' Hughes et al. (in prep) based on data from Iacono-Marziano et al. (2010)
    - Ar_Rhyolite_HughesIP: Hughes et al. (in prep) based on data from Iacono-Marziano et al. (2010)
    - Ne_Basalt_HughesIP: Hughes et al. (in prep) based on data from Iacono-Marziano et al. (2010)
    - Ne_Rhyolite_HughesIP: Hughes et al. (in prep) based on data from Iacono-Marziano et al. (2010)
    - [user specified number]: User can type a number that will be used instead (i.e., a constant value)

## Python set-up

In [1]:
import pandas as pd
import VolFe as vf
import math
import matplotlib as plt

## Composition and conditions used for benchmarking

Average of high-SiO2 pillow-rim glasses in HSDP from Mauna Kea volcano from Brounce et al. (2017).

In [2]:
my_analysis = {
        "Sample": "Hawaiian basalt",
        "T_C": 1200.0,  # Temperature in 'C
        "SiO2": 51.29,  # wt%
        "TiO2": 2.50,  # wt%
        "Al2O3": 13.70,  # wt%
        "FeOT": 11.04,  # wt%
        "MnO": 0.02,  # wt%
        "MgO": 6.70,  # wt%
        "CaO": 11.03,  # wt%
        "Na2O": 2.27,  # wt%
        "K2O": 0.43,  # wt%
        "P2O5": 0.,  # wt%
        "H2O": 0.,  # wt%
        "CO2ppm": 0.,  # ppm
        "STppm": 0.,  # ppm
        "Xppm": 0.0,  # ppm
        "Fe3FeT": 0.1}

my_analysis = pd.DataFrame(my_analysis, index=[0])

PT = {"P":1000.}
PT["T"]=1200.

melt_wf=vf.melt_comp(0.,my_analysis)
melt_wf['CO2'] = my_analysis.loc[0.,"CO2ppm"]/1000000.
melt_wf["H2OT"] = my_analysis.loc[0,"H2O"]/100.
melt_wf['ST'] = my_analysis.loc[0.,"STppm"]/1000000.
melt_wf['CT'] = (melt_wf['CO2']/vf.species.loc['CO2','M'])*vf.species.loc['C','M']
melt_wf['HT'] = (melt_wf['H2OT']/vf.species.loc['H2O','M'])*(2.*vf.species.loc['H','M'])
melt_wf['XT'] = my_analysis.loc[0.,"Xppm"]/1000000.
melt_wf["Fe3FeT"] = my_analysis.loc[0.,"Fe3FeT"]

## Models for solubility function of sulfide (*S2-)

option = sulfide, function = C_S

The following models do not have material available in the original papers for benchmarking:
- 'Boulliung23eq6' Eq. (6) from Boulliung & Wood (2023) CMP 178:56 doi:10.1007/s00410-023-02033-9

### 'ONeill21' Eq. (10.34) from O'Neill (2021)

Supplementary spreadsheet (sulfide,SCSS=ONeill21.xlsx)

Cell AN14: -2.419

Matches to 3 decimal places: Note spreadsheet uses +273 to convert to K, rather than 273.15 used in VolFe so T in spreadsheet = 1200.15 'C

In [ ]:
my_models = [["sulfide", "ONeill21"]]
my_models = vf.make_df_and_add_model_defaults(my_models)
math.log(vf.C_S(PT,melt_wf,models=my_models))

### 'Boulliung23eq7' Equation (7) from Boulliung et al. (2023)
Supplementary Information - Supplementary file1 used for benchmarking (sulfide=Boulliung23eq7_sulfate=Boulliung23eq9,Boulliung23eq11.xlsx)

Cell K22: -5.1286877

Matches to 3 decimal places

In [ ]:
my_models = [["sulfide", "Boulliung23eq7"]]
my_models = vf.make_df_and_add_model_defaults(my_models)
math.log10(vf.C_S(PT,melt_wf,models=my_models))

## Models for solubility function of sulfate (S6+)

option = sulfate, function = C_SO4

The following models do not have material available in the original papers for benchmarking:
- Eq. (5) from Boulliung & Wood (2023) GCA 343:420 doi:10.1016/j.gca.2022.11.025 and Eq. (8) for P from Boulliung & Wood (2022)

### 'ONeill22' Eq. (12a) without H2O dilution from O'Neill & Mavrogenes (2022)
Appendix A. Supplementary material - Supplementary data 2. Tab = Table S6 S redox calculator (sulfate,KSOg2=ONeill22.xlsx),

Cell AH12: 12.95

Matches to 2 decimal places. Note spreadsheet uses +273 to convert to K, rather than 273.15 used in VolFe so T in spreadsheet = 1200.15 'C

In [3]:
my_models = [["sulfate", "ONeill22"],["KOSg2", "ONeill22"]]
my_models = vf.make_df_and_add_model_defaults(my_models)
math.log(vf.C_SO4(PT,melt_wf,models=my_models)/(vf.KOSg2(PT,models=my_models)))

12.954840211020695

### 'Boulliung22nP' Equation (5) from Boulliung et al. (2022) TO DO

### 'Boulliung23eq9' Equation (9) from Boulliung et al. (2023)
Supplementary Information - Supplementary file1 used for benchmarking (sulfide=Boulliung23eq7_sulfate=Boulliung23eq9,Boulliung23eq11.xlsx)

Cell K24: 8.73571472

Matches to 2 decimal places

In [ ]:
my_models = [["sulfate", "Boulliung23eq9"]]
my_models = vf.make_df_and_add_model_defaults(my_models)
math.log10(vf.C_SO4(PT,melt_wf,models=my_models))

### 'Boulliung23eq11' Equation (11) from Boulliung et al. (2023)
Supplementary Information - Supplementary file1 used for benchmarking (sulfide=Boulliung23eq7_sulfate=Boulliung23eq9,Boulliung23eq11.xlsx)

Cell M24: 8.82627819

Matches to 3 decimal places

In [ ]:
my_models = [["sulfate", "Boulliung23eq11"]]
my_models = vf.make_df_and_add_model_defaults(my_models)
math.log10(vf.C_SO4(PT,melt_wf,models=my_models))

## Models for equilibrium functions of CO2 speciation

option = Cspeccomp, function = KCOm

'Basalt' and 'Rhyolite' are single values and are therefore not benchmarked.

### 'Andesite_Botcharnikov06' Eq. (8) from Botcharnikov et al. (2006)

### 'Dacite_Botcharnikov06' Eq. in the text from Botcharnikov et al. (2006), based on data from Behrens et al. (2004)

Fig. 12 from Botcharnikov et al. (2006) shows the fit to the data used to derive their models for both Andesite and Dacite (see Cspeccomp=Andesite_Botcharnikov06,Dacite_Botcharnikov06 [Fig12_Botcharnikov06].jpg).

As they are straight lines, we read the data at the intersection of the axes

Andesite: 

Intersection with y-axis: 1000/T = 0.600 and lnK2 = 0.20

Intersection with x-axis: 1000/T = 1.405 and lnK2 = 7.00

Dacite:

Intersection with y-axis: 1000/T = 0.600 and lnK2 = 1.85

Intersection with x-axis: 1000/T = 1.500 and lnK2 = 7.00

And then calculate at the same temperatures using VolFe - there is good agreement for both models.

In [ ]:
my_models = [["Cspeccomp", "Andesite_Botcharnikov06"]]
my_models = vf.make_df_and_add_model_defaults(my_models)

PT = {"T":(1000./0.6)-273.15}
T1 = math.log(vf.KCOm(PT,melt_wf,models=my_models))
PT = {"T":(1000./1.405-273.15)}
T2 = math.log(vf.KCOm(PT,melt_wf,models=my_models))

print(T1, T2)

In [ ]:
my_models = [["Cspeccomp", "Dacite_Botcharnikov06"]]
my_models = vf.make_df_and_add_model_defaults(my_models)

PT = {"T":(1000./0.6)-273.15}
T1 = math.log(vf.KCOm(PT,melt_wf,models=my_models))
PT = {"T":(1000./1.5)-273.15}
T2 = math.log(vf.KCOm(PT,melt_wf,models=my_models))

print(T1, T2)